In [3]:
import requests

In [4]:
url = "https://www.1-chome.com/api/keitai/listPage"  # 在Network里找到
headers = {
    "User-Agent": "Mozilla/5.0 ...",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://example.com/products",
    # 如果接口需要认证/CSRF，按Network里复制的为准
    # "Authorization": "Bearer ...",
    # "Cookie": "session=....",
}
params = {"page":"1","size":"24","keyword":"","isImpo":"false","isCampaign":"false","cateCode":"RGNg976kptBN7UjF","kbNames":"","isImpoCate":"true"}  # 如果Network里有查询参数
resp = requests.get(url, headers=headers, params=params, timeout=15)
resp.raise_for_status()
data = resp.json()  # 直接是JSON最稳
print(data)

{'code': 200, 'msg': 'success', 'data': {'totalElements': 28, 'totalPages': 2, 'pageable': {'pageNumber': 0, 'pageSize': 24, 'sort': {'sorted': False, 'empty': True, 'unsorted': True}, 'offset': 0, 'paged': True, 'unpaged': False}, 'size': 24, 'content': [{'goodsId': 1374, 'cateCode': 'WWwbdyaQll11eiSM', 'title': 'iPhone 17 Pro 256GB', 'description': None, 'impo': 1, 'sort': 20, 'impoSort': None, 'listImageUrl': 'bd4b392c-8b8f-4ab7-8156-3efbdb7739ac.png', 'isKeitaiItem': True, 'carrier': None, 'maker': None, 'smartphoneId': None, 'price': None, 'jan': None, 'allGoodsKbId': 1936, 'kbName': '新品', 'kbCountPerAppli': 30, 'kbCount': 95, 'hasLimit': True, 'kbDesc': '', 'isFavorite': False, 'accountFavoriteId': None, 'goodsKbDetails': [{'allGoodsKbDetailId': 3013, 'kbDetailName': '未開封', 'kbDetailPrice': 177000, 'maxPrice': 177000, 'maxCamPrice': None, 'noConfirm': True}, {'allGoodsKbDetailId': 3014, 'kbDetailName': '開封済未使用品', 'kbDetailPrice': 162000, 'maxPrice': 162000, 'maxCamPrice': None, '

In [6]:


for i in range(len(data['data']['content'])):
    print(data['data']['content'][i]['title'])
    # print(data['data']['content'][i]['goodsKbDetails'])

    for j in range(len(data['data']['content'][i]['goodsKbDetails'])):
        print(data['data']['content'][i]['goodsKbDetails'][j]['kbDetailName'],"-",data['data']['content'][i]['goodsKbDetails'][j]['kbDetailPrice'])
        print()


iPhone 17 Pro 256GB
未開封 - 177000

開封済未使用品 - 162000

iPhone 17 Pro 512GB
未開封 - 212000

開封済未使用品 - 197000

iPhone 17 Pro 1TB
未開封 - 247000

開封済未使用品 - 230000

iPhone 17 Pro Max 256GB
未開封 - 205000

開封済未使用品 - 185000

iPhone 17 Pro Max 512GB
未開封 - 240000

開封済未使用品 - 220000

iPhone 17 Pro Max 1TB
未開封 - 270000

開封済未使用品 - 258000

iPhone 17 Pro Max 2TB
未開封 - 330000

開封済未使用品 - 305000

iPhone Air 256GB
未開封 - 161000

開封済未使用品 - 145000

iPhone Air 512GB
未開封 - 194000

開封済未使用品 - 178000

iPhone Air 1TB
未開封 - 227000

開封済未使用品 - 215000

iPhone 17 256GB
未開封 - 120000

開封済未使用品 - 111000

iPhone 17 512GB
未開封 - 153000

開封済未使用品 - 140000

iPhone 16 Pro 128GB
未開封 - 153000

開封済未使用 - 138000

iPhone 16 Pro 256GB
未開封 - 172500

開封済未使用 - 157000

iPhone 16 Pro 512GB
未開封 - 199500

開封済未使用 - 185000

iPhone 16 Pro 1TB
未開封 - 217500

開封済未使用 - 202500

iPhone 16 Pro Max 256GB
未開封 - 180500

開封済未使用 - 168000

iPhone 16 Pro Max 512GB
未開封 - 209500

開封済未使用 - 197000

iPhone 16 Pro Max 1TB
未開封 - 240500

開封済未使用 - 223000

iPhone 16 128GB
未開封 

In [1]:
# from playwright.sync_api import sync_playwright
#
# def main():
#     with sync_playwright() as p:
#         # 启动浏览器（headless=False 表示可见模式）
#         browser = p.chromium.launch(headless=False, slow_mo=500)
#         page = browser.new_page()
#
#         # 打开目标网页
#         page.goto("https://www.mobile-ichiban.com/Prod/1/01")
#
#         # 点击某个元素（示例：页面上的链接）
#         # page.click("a")  # 用 CSS 选择器定位元素，这里点击第一个 <a> 标签
#
#         # 保持浏览器打开，等待你手动关闭
#         input("按回车键后关闭浏览器...")
#         browser.close()

In [1]:
import secrets
print(secrets.token_urlsafe(16))   # ≈22字符，URL安全
print(secrets.token_urlsafe(24))   # ≈32字符，URL安全（更强）
print(secrets.token_hex(16))

7kgVlcMczUO46Oimg4msuw
XNgCZCQN7dvkSP7K17xmK8aq-6_bjvI_
0ff8a581e3688826f0949d36927a5d9a


In [2]:
import requests
base_url = "https://www.apple.com/jp/shop/fulfillment-messages"


part_number =[
    "MG674J/A",
"MG684J/A",
"MG694J/A",
"MG6A4J/A",
"MG6C4J/A",
"MG6D4J/A",
"MG6E4J/A",
"MG6F4J/A",
"MG6G4J/A",
"MG6H4J/A",
"MG854J/A",
"MG864J/A",
"MG874J/A",
"MG894J/A",
"MG8A4J/A",
"MG8C4J/A",
"MG8D4J/A",
"MG8E4J/A",
"MG8F4J/A",
"MFY84J/A",
"MFY94J/A",
"MFYA4J/A",
"MFYC4J/A",
"MFYD4J/A",
"MFYE4J/A",
"MFYF4J/A",
"MFYG4J/A",
"MFYH4J/A",
"MFYJ4J/A",
"MFYK4J/A",
"MFYL4J/A",
"MG274J/A",
"MG284J/A",
"MG294J/A",
"MG2A4J/A",
"MG2C4J/A",
"MG2D4J/A",
"MG2E4J/A",
"MG2F4J/A",
"MG2G4J/A",
"MG2H4J/A",
"MG2J4J/A",
"MG2K4J/A"
]


for i in part_number:
    params = {
    "fae": "true",
    "pl": "true",
    "mts.0": "regular",
    "mts.1": "compact",
    "cppart": "UNLOCKED_JP",
    "parts.0": i,
    "searchNearby": "true",
    "store": "R718"
}
    req = requests.Request('GET', base_url, params=params)
    prepped = req.prepare()
    print(prepped.url)

# 准备请求而不发送

# 输出完整 URL


https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MG674J%2FA&searchNearby=true&store=R718
https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MG684J%2FA&searchNearby=true&store=R718
https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MG694J%2FA&searchNearby=true&store=R718
https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MG6A4J%2FA&searchNearby=true&store=R718
https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MG6C4J%2FA&searchNearby=true&store=R718
https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MG6D4J%2FA&searchNearby=true&store=R718
https://www.apple.com/

In [49]:
import pandas as pd
import os,re
from pathlib import Path
from typing import Protocol, Dict, Callable, Optional,List


# AppleStockChecker/utils/external_ingest/helpers.py
from __future__ import annotations
import io, re
from typing import Optional, Tuple
import httpx
import pandas as pd
from django.utils import timezone
from django.utils.dateparse import parse_datetime, parse_date



class Cleaner(Protocol):
    def __call__(self, df: pd.DataFrame) -> pd.DataFrame: ...


HTTP_TIMEOUT = 30.0
_YEN_RE = re.compile(r"[^\d]+")
_CAP_RE = re.compile(r"(\d+)\s*(TB|GB)", re.IGNORECASE)
_NUM_MODEL_PAT = re.compile(r"(iPhone)\s*(\d{2})(?:\s*(Pro\s*Max|Pro|Plus|mini))?", re.I)
_AIR_PAT = re.compile(r"(iPhone)\s*(Air)(?:\s*(Pro\s*Max|Pro|Plus|mini))?", re.I)
_PN_REGEX = re.compile(r"\b[A-Z0-9]{4,6}\d{0,3}J/A\b")


CLEANERS: Dict[str, Cleaner] = {}

def register_cleaner(name: str):
    def deco(fn: Cleaner):
        CLEANERS[name] = fn
        return fn
    return deco

PN_REGEX = re.compile(r"\b[A-Z0-9]{4,6}\d{0,3}J/A\b")

def _parse_yen(val) -> int | None:
    """'¥177,000' / '177,000円' / '177000' -> 177000"""
    if val is None: return None
    s = str(val).strip()
    if not s: return None
    s = _YEN_RE.sub("", s)
    if not s: return None
    try:
        n = int(s)
        return n
    except Exception:
        return None


async def async_http_get_bytes(url: str, *, headers: dict | None = None) -> bytes:
    async with httpx.AsyncClient(timeout=HTTP_TIMEOUT, follow_redirects=True) as client:
        r = await client.get(url, headers=headers or {})
        r.raise_for_status()
        return r.content


def read_csv_smart(data: bytes, *, encodings=("utf-8-sig", "utf-8", "cp932", "shift_jis"), seps=(",", "\t", ";", "|")) -> pd.DataFrame:
    # 先用 c 引擎快读，不行再用 python 引擎
    for enc in encodings:
        for sep in seps:
            try:
                df = pd.read_csv(io.BytesIO(data), encoding=enc, sep=sep, engine="c", on_bad_lines="skip")
                if df.shape[1] == 1 and any(x in str(df.columns[0]) for x in [",", "\t", ";", "|"]):
                    raise ValueError("wrong sep")
                return df
            except Exception:
                pass
    for enc in encodings:
        for sep in seps:
            try:
                return pd.read_csv(io.BytesIO(data), encoding=enc, sep=sep, engine="python", on_bad_lines="skip")
            except Exception:
                pass
    # 兜底：按 Excel 读（万一远端返回的是 xlsx）
    try:
        return pd.read_excel(io.BytesIO(data))
    except Exception as e:
        raise RuntimeError(f"无法解析为 CSV/Excel: {e}")


def to_int_yen(s: object) -> Optional[int]:
    if s is None: return None
    txt = str(s).strip()
    if not re.search(r"\d", txt): return None
    # 范围 "105,000～110,000"
    parts = re.split(r"[~～\-–—]", txt)
    candidates = []
    for p in parts:
        # 排除 12-14 位纯数字（像 JAN/电话）
        if re.fullmatch(r"\d{12,14}", p.strip()):
            continue
        digits = re.sub(r"[^\d万]", "", p)
        if not digits:
            continue
        if "万" in digits:
            m = re.search(r"([\d\.]+)万", digits)
            base = float(m.group(1)) if m else 0.0
            candidates.append(int(base * 10000))
        else:
            candidates.append(int(re.sub(r"[^\d]", "", digits)))
    if not candidates:
        return None
    val = max(candidates)
    # 合理区间过滤
    if val < 1000 or val > 5_000_000:
        return None
    return val


def parse_dt_aware(s: object) -> timezone.datetime:
    if not s:
        return timezone.now()
    txt = str(s).strip()
    dt = parse_datetime(txt)
    if dt is None:
        d = parse_date(txt)
        if d:
            dt = timezone.datetime(d.year, d.month, d.day)
    if dt is None:
        return timezone.now()
    if timezone.is_naive(dt):
        dt = timezone.make_aware(dt, timezone.get_current_timezone())
    return dt


def pick_first_col(df: pd.DataFrame, *candidates: str) -> pd.Series:
    """
    在 df 里按顺序返回第一列；都没有则返回空 Series。
    用它替代 df.get('A') or df.get('B') 这种写法，避免 Series 布尔判断歧义。
    """
    for c in candidates:
        if c in df.columns:
            return df[c]
    return pd.Series(dtype=object)


def _normalize_model_generic(text: str) -> str:
    """
    统一型号主体：
      - iPhone17/16 + 后缀（Pro/Pro Max/Plus/mini）
      - iPhone Air（含“17 air”→ Air）
      - 允许紧凑写法：17pro / 17promax / 16Pro / 16Plus ...
    输出：'iPhone 17 Pro Max' / 'iPhone 17 Pro' / 'iPhone Air' / ...
    """
    if not text:
        return ""
    t = str(text).replace("\u3000", " ")
    t = re.sub(r"\s+", " ", t)

    # 日文别名到英文
    t = (t.replace("プロマックス", "Pro Max")
           .replace("プロ", "Pro")
           .replace("プラス", "Plus")
           .replace("ミニ", "mini")
           .replace("エアー", "Air")
           .replace("エア", "Air"))

    # ❗ 在“数字后立即跟英文”的位置补一个空格：17pro -> 17 pro
    t = re.sub(r"(\d{2})(?=[A-Za-z])", r"\1 ", t)

    # 标准化大小写/形态：pro-max / ProMax / promáx → Pro Max；pro → Pro；plus → Plus；mini → mini
    t = re.sub(r"(?i)\bpro\s*max\b", "Pro Max", t)
    t = re.sub(r"(?i)\bpro\b", "Pro", t)
    t = re.sub(r"(?i)\bplus\b", "Plus", t)
    t = re.sub(r"(?i)\bmini\b", "mini", t)

    # 若没有 iPhone 前缀但出现纯数字代号，补上
    if "iPhone" not in t and re.search(r"\b1[0-9]\b", t):
        t = re.sub(r"\b(1[0-9])\b", r"iPhone \1", t, count=1)

    # 特例：'17 air' → iPhone Air（防止被当成 iPhone 17）
    t = re.sub(r"(?i)\biPhone\s+17\s+Air\b", "iPhone Air", t)

    # 去容量/SIM/括号噪声
    t = re.sub(r"(\d+(?:\.\d+)?\s*TB|\d{2,4}\s*GB)", "", t, flags=re.I)
    t = re.sub(r"SIMフリ[ーｰ–-]?|シムフリ[ーｰ–-]?|sim\s*free", "", t, flags=re.I)
    t = re.sub(r"[（）\(\)\[\]【】].*?[（）\(\)\[\]【】]", "", t)
    t = re.sub(r"\s+", " ", t).strip()

    # 1) 数字代号机型
    m = _NUM_MODEL_PAT.search(t)
    if m:
        base = f"{m.group(1)} {m.group(2)}"
        suf  = (m.group(3) or "").strip()
        return f"{base} {suf}".strip()

    # 2) Air
    m2 = _AIR_PAT.search(t)
    if m2:
        # 当前返回主体 'iPhone Air'；若以后真有 Air Plus 等可在此扩展
        return "iPhone Air"

    return ""


def _parse_capacity_gb(text: str) -> Optional[int]:
    if not text:
        return None
    t = str(text)
    m = re.search(r"(\d+(?:\.\d+)?)\s*TB", t, flags=re.I)
    if m:
        return int(round(float(m.group(1)) * 1024))
    m = re.search(r"(\d{2,4})\s*GB", t, flags=re.I)
    if m:
        return int(m.group(1))
    return None

def _resolve_iphone17_info_path() -> str:
    """
    解析 iphone17_info.csv 的路径：
      1) settings.EXTERNAL_IPHONE17_INFO_PATH
      2) 环境变量 IPHONE17_INFO_CSV
      3) 项目内默认 AppleStockChecker/data/iphone17_info.csv
    """
    # 1) settings 覆盖
    try:
        from django.conf import settings
        p = getattr(settings, "EXTERNAL_IPHONE17_INFO_PATH", None)
        if p:
            return str(p)
    except Exception:
        pass

    # 2) 环境变量
    envp = os.getenv("IPHONE17_INFO_CSV")
    if envp and Path(envp).exists():
        return envp

    # 3) 项目内默认
    # 计算当前文件所在 app 的 data 目录
    here = Path(__file__).resolve()
    app_root = here.parents[2]  # .../AppleStockChecker/
    default_path = app_root / "data" / "iphone17_info.csv"
    return str(default_path)

def _load_iphone17_info_df() -> pd.DataFrame:
    """
    读取 AppleStockChecker/data/iphone17_info.csv 或 settings / env 指定的路径。
    输出列：part_number, model_name_norm, capacity_gb
    """
    # 解析路径（settings > env > 默认）
    # try:
    #     from django.conf import settings
    #     p = getattr(settings, "EXTERNAL_IPHONE17_INFO_PATH", None)
    #     if p:
    #         path = str(p)
    #     else:
    #         raise AttributeError
    # except Exception:
    #     path = os.getenv("IPHONE17_INFO_CSV") or str(Path(__file__).resolve().parents[2] / "data" / "iphone17_info.csv")

    path = "/Users/syu/PycharmProjects/YamagotiProjects/AppleStockChecker/data/iphone17_info.csv"
    pth = Path(path)
    if not pth.exists():
        raise FileNotFoundError(f"未找到 iphone17_info：{pth}")

    if re.search(r"\.(xlsx|xlsm|xls|ods)$", str(pth), re.I):
        df = pd.read_excel(pth)
    else:
        df = pd.read_csv(pth, encoding="utf-8-sig")

    need = {"part_number", "model_name", "capacity_gb","color"}
    missing = need - set(df.columns)
    if missing:
        raise ValueError(f"iphone17_info 缺少必要列：{missing}")

    df = df.copy()
    df["model_name_norm"] = df["model_name"].map(_normalize_model_generic)
    df["capacity_gb"] = pd.to_numeric(df["capacity_gb"], errors="coerce").astype("Int64")
    df = df.dropna(subset=["model_name_norm", "capacity_gb", "part_number"])
    return df[["part_number", "model_name_norm", "capacity_gb"]]

def _load_iphone17_info_df_for_shop2() -> pd.DataFrame:
    """
    读取 AppleStockChecker/data/iphone17_info.csv 或 settings / env 指定的路径。
    输出列：part_number, model_name_norm, capacity_gb
    """
    path = "/Users/syu/PycharmProjects/YamagotiProjects/AppleStockChecker/data/iphone17_info.csv"
    pth = Path(path)
    if not pth.exists():
        raise FileNotFoundError(f"未找到 iphone17_info：{pth}")

    if re.search(r"\.(xlsx|xlsm|xls|ods)$", str(pth), re.I):
        df = pd.read_excel(pth)
    else:
        df = pd.read_csv(pth, encoding="utf-8-sig")

    need = {"part_number", "model_name", "capacity_gb","color"}
    missing = need - set(df.columns)
    if missing:
        raise ValueError(f"iphone17_info 缺少必要列：{missing}")

    df = df.copy()
    # df["model_name_norm"] = df["model_name"].map(_normalize_model_generic)
    df["capacity_gb"] = pd.to_numeric(df["capacity_gb"], errors="coerce").astype("Int64")
    df = df.dropna(subset=["model_name", "capacity_gb", "part_number","color"])
    return df[["part_number", "model_name", "capacity_gb","color"]]


def _clean_text(x: object) -> str:
    s = "" if x is None else str(x)
    s = s.replace("\u3000", " ")          # 全角空格
    s = s.replace("\r", " ").replace("\n", " ")  # 去换行
    return re.sub(r"\s+", " ", s).strip()

def _extract_part_number(text: str) -> str | None:
    t = _clean_text(text)
    # 1) 优先：显式 “型番: XXXXXJ/A”
    m = re.search(r"型番[:：]\s*([A-Z0-9]{4,6}\d{0,3}J/A)\b", t)
    if m:
        return m.group(1)
    # 2) 兜底：全文 PN 正则
    m2 = PN_REGEX.search(t)
    return m2.group(0) if m2 else None


def _extract_price_new(raw: object) -> Optional[int]:
    """
    从 'price' 字段取整数日元：
      - 去掉'新品'等字样与货币符号
      - 支持区间 '82,000～84,000' 取最大
      - 合理区间过滤由 to_int_yen 完成
    """
    if raw is None:
        return None
    s = str(raw)
    # 去掉“新品”等修饰词
    s = s.replace("新品", "").replace("新\u54c1", "")
    # 常见前缀也清理一下
    s = s.replace("未開封", "").replace("未开封", "")
    return to_int_yen(s)

def _price_from_shop7(x: object) -> Optional[int]:
    """data2 -> price_new：去掉“新品/未開封/货币符号/逗号”，区间取最大"""
    if x is None:
        return None
    s = str(x)
    s = s.replace("新品", "").replace("新\u54c1", "")
    s = s.replace("未開封", "").replace("未开封", "")
    return to_int_yen(s)

def _norm_model_for_shop7(text: str) -> str:
    """
    在 _normalize_model_generic 之前做一点“shop7 特有”的宽松处理：
      - ‘promax/ProMax/pro-max’ → ‘Pro Max’
      - ‘17 air’ → ‘iPhone Air’
      - 没有 iPhone 前缀但有 '17' 的，补成 ‘iPhone 17 ...’
    然后交给 _normalize_model_generic 做最终归一。
    """
    if not text:
        return ""
    t = str(text).replace("\u3000", " ")
    t = re.sub(r"\s+", " ", t)

    # 日文/英文后缀标准化
    t = (t.replace("プロマックス", "Pro Max")
           .replace("プロ", "Pro")
           .replace("プラス", "Plus")
           .replace("ミニ", "mini")
           .replace("エアー", "Air")
           .replace("エア", "Air"))

    # promax 连写/大小写
    t = re.sub(r"(?i)pro[-\s]?max", "Pro Max", t)

    # 若没有 iPhone 前缀但出现 "17 air" / "17 pro max" / "17 pro" / "17 plus"
    # 先把 "17 air" 显式改成 "iPhone Air"（Air 没有数字后缀）
    if re.search(r"(?i)\b17\s+air\b", t):
        # 去掉“17 ”，以免 _normalize_model_generic 误识别为 iPhone 17
        t = re.sub(r"(?i)\b17\s+air\b", "iPhone Air", t)

    # 若没有 iPhone 单词但有纯数字代号（例如 "17 Pro Max 256GB"）
    if "iPhone" not in t and re.search(r"\b1[0-9]\b", t):
        t = re.sub(r"\b(1[0-9])\b", r"iPhone \1", t, count=1)

    return _normalize_model_generic(t)

def _resolve_info_path() -> Path:
    try:
        from django.conf import settings
        p = getattr(settings, "EXTERNAL_IPHONE17_INFO_PATH", None)
        if p:
            return Path(p)
    except Exception:
        pass
    envp = os.getenv("IPHONE17_INFO_CSV")
    if envp and Path(envp).exists():
        return Path(envp)
    return Path(__file__).resolve().parents[2] / "data" / "iphone17_info.csv"


def _load_jan_to_pn() -> Dict[str, str]:
    """
    返回 { jan(13位字符串) : part_number } 的字典。
    若 info 文件没有 jan 列，则返回空字典（后续走 data8 的 PN 兜底）。
    """
    path = _resolve_info_path()
    if not path.exists():
        # 没找到映射文件时，仍允许仅走 data8 的 PN 兜底
        return {}
    if re.search(r"\.(xlsx|xlsm|xls|ods)$", str(path), re.I):
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path, encoding="utf-8-sig")

    if "part_number" not in df.columns:
        # 没有 PN 列，无法映射
        return {}

    # 允许 info 表没有 jan；有则清洗为 13 位
    if "jan" in df.columns:
        df = df.copy()
        df["jan"] = df["jan"].astype(str).str.replace(r"[^\d]", "", regex=True)
        df = df[df["jan"].str.fullmatch(r"\d{13}", na=False)]
        mapping = dict(zip(df["jan"].astype(str), df["part_number"].astype(str)))
        return mapping
    return {}

def _extract_pn_from_text(text: object) -> Optional[str]:
    if text is None:
        return None
    s = str(text).replace("\u3000", " ")
    m = _PN_REGEX.search(s)
    return m.group(0) if m else None

def _price_from_shop6_data7(x: object) -> Optional[int]:
    if x is None:
        return None
    s = str(x)
    # 去一些修饰词
    s = (s.replace("新品", "")
           .replace("新\u54c1", "")
           .replace("未開封", "")
           .replace("未开封", ""))
    return to_int_yen(s)


def _extract_jan_from_data(x: object) -> Optional[str]:
    """
    从 'data' 文本里抽取 13 位 JAN（例如 'JAN:4549995648300'）
    """
    if x is None:
        return None
    s = str(x)
    # 优先匹配 'JAN:XXXXXXXXXXXXX'
    m = re.search(r"JAN[:：]?\s*(\d{13})", s)
    if m:
        return m.group(1)
    # 兜底：任意 13 位数字
    m2 = re.search(r"\b(\d{13})\b", s)
    return m2.group(1) if m2 else None

def _price_from_shop5(x: object) -> Optional[int]:
    """
    price 列 -> price_new：
      - 去掉前导 '～'、'新品'、'未開封' 等修饰
      - 支持区间 '～162,500円' / '162,500～170,000円'：取最大值
      - 支持 '10.5万'
    """
    if x is None:
        return None
    s = str(x).strip()
    s = s.lstrip("～")  # '～162,500円' -> '162,500円'
    s = (s.replace("新品", "")
           .replace("新\u54c1", "")
           .replace("未開封", "")
           .replace("未开封", ""))
    return to_int_yen(s)

def _price_from_shop3(x: object) -> Optional[int]:
    """
    data5 -> price_new
    - 预期形如 '¥177,000'；也兼容 '～177,000円'/'10.5万' 等；取区间最大值
    - 去除可能出现的修饰词（“新品/未開封”等）
    """
    if x is None:
        return None
    s = str(x)
    s = (s.replace("新品", "")
           .replace("新\u54c1", "")
           .replace("未開封", "")
           .replace("未开封", ""))  # 安全冗余
    return to_int_yen(s)


def _norm(s: str) -> str:
    return (s or "").strip()

def _norm_model_token(s: str) -> str:
    """
    将 data2-1 的机型片段“宽松”规范化（仅用于和 iphone17_info 里的 model_name 做宽松匹配）
    规则：小写、去空格、去多余符号
    """
    s = (s or "").lower()
    s = re.sub(r"iphone\s*", "iphone ", s)
    s = re.sub(r"[^0-9a-z\s+]", "", s)  # 仅保留 a-z0-9 和空格
    s = re.sub(r"\s+", " ", s).strip()
    return s

def _pick_model_name_loose(model_token: str, iphone17_df: pd.DataFrame) -> str | None:
    """
    宽松匹配：在 iphone17_df['model_name'] 中选与 token 最匹配的项（不严格 Fuzzy，先简单包含匹配）
    """
    token = _norm_model_token(model_token)
    if not token: return None
    # 候选（去重）
    candidates = list(dict.fromkeys([_norm(x) for x in iphone17_df["model_name"].dropna().tolist()]))
    # 简单策略：同样规范化后，包含则命中
    def norm_m(m): return _norm_model_token(m)
    hits = [m for m in candidates if token in norm_m(m) or norm_m(m) in token]
    if len(hits) == 1:
        return hits[0]
    # 多命中时偏向更长的 model_name（更具体）
    if hits:
        return sorted(hits, key=lambda m: len(m), reverse=True)[0]
    return None

def _parse_adjust_rule(s: str) -> dict:
    """
    解析 data5 的减价规则：
      '青-1000' → {'青': -1000}
      '銀-5000+++青-5000' → {'銀': -5000, '青': -5000}
    返回：{组名: 负数(或0)}
    """
    rules = {}
    if not s: return rules
    # 允许 '+++', '+', '，' 等作为分隔
    parts = re.split(r"\+{1,3}|[,、，\s]+", str(s))
    for p in parts:
        p = p.strip()
        if not p: continue
        m = re.match(r"(.+?)-(\d+)", p)
        if not m:
            continue
        group = m.group(1).strip()
        amt = -int(m.group(2))
        rules[group] = amt
    return rules

def _apply_adjust_for_colorname(color_name: str, rules: dict) -> int:
    """
    根据规则返回针对该“颜色名”的价格修正（和机型容量下实际存在的颜色匹配）。
    约定：
      - '青'：匹配包含「ブルー」的颜色（ミストブルー/ディープブルー/スカイブルー 等）
      - '銀'/'シルバー'：匹配包含「シルバー」
      - 可扩展其它组（例：'黒'->「ブラック」；'白'->「ホワイト/シルバー」等）
    """
    c = color_name or ""
    adjust = 0
    for group, delta in rules.items():
        g = group.strip()
        if g in ("青", "ブルー","ミストブルー","ディープブルー","スカイブルー"):
            if "ブルー" in c:
                adjust += delta
        elif g in ("銀", "シルバー",):
            if "シルバー" in c:
                adjust += delta
        else:
            # 精确匹配 group 文字（万一 data5 直接写具体颜色）
            if g and g in c:
                adjust += delta
    return adjust


def clean_shop2(shop2_df: pd.DataFrame) -> pd.DataFrame:
    """
    输入：
      - shop2_df: 读取自 shop2.csv（columns: web-scraper-order, web-scraper-start-url, data2-1, data2-2, ..., data5, ..., data3, time-scraped）
      - iphone17_df: 读取自 iphone17_info.csv（至少包含: model_name, capacity_gb, color, part_number）
    输出 DataFrame 列：
      - part_number, shop_name, price_new, recorded_at
    规则：
      - 仅 data2-2 含 'simfree' 且含 '未開封'（且不含 '開封'）的行
      - data2-1 解析机型+容量；若在 iphone17_df 找不到对应机型容量 → 跳过
      - 价格 data3；data5 减价规则（青/銀等组）会作用到对应颜色（蓝系/银系）
      - shop_name 固定 '海峡通信'；recorded_at = time-scraped
    """
    SHOP = "海峡通信"

    # 统一列名（小写）
    df = shop2_df.copy()
    df.columns = [c.strip().lower() for c in df.columns]

    # 必要列存在性检查
    need_cols = ["data2-1","data2-2","data3","data5","time-scraped"]
    for c in need_cols:
        if c not in df.columns:
            df[c] = None

    # 只保留 simfree 未開封
    def _is_target(s: str) -> bool:
        s = (s or "").lower()
        return ("simfree" in s) and ("未開封" in s)

    df = df[df["data2-2"].apply(_is_target)]
    if df.empty:
        return pd.DataFrame(columns=["part_number","shop_name","price_new","recorded_at"])

    # iphone17_df 预处理
    info = _load_iphone17_info_df_for_shop2()
    # info = iphone17_df.copy()
    # info["model_name"] = info["model_name"].apply(_norm)
    # 容量转 int GB
    if "capacity_gb" not in info.columns:
        # 如果你的 info 表容量列叫别的名字，替换这里
        raise ValueError("iphone17_info.csv 需要包含 capacity_gb 列")
    # 颜色规范
    info["color"] = info["color"].apply(_norm)

    out_rows = []

    for _, row in df.iterrows():
        raw_modelcap = _norm(row.get("data2-1"))
        if not raw_modelcap:
            continue

        # 容量
        cap_gb = _parse_capacity_gb(raw_modelcap)
        if not cap_gb:
            continue

        # 机型（宽松匹配）
        model_name = _pick_model_name_loose(raw_modelcap, info)
        if not model_name:
            continue

        # 该机型容量下的所有颜色
        sub = info[(info["model_name"] == model_name) & (info["capacity_gb"] == cap_gb)].copy()
        if sub.empty:
            continue

        # 基础价格
        base_price = _parse_yen(row.get("data3"))
        if base_price is None:
            continue

        # 减价规则
        rules = _parse_adjust_rule(row.get("data5"))


        # 记录时间
        rec_raw = row.get("time-scraped")
        # 容忍多种日期格式
        try:
            rec_dt = pd.to_datetime(rec_raw, utc=True, errors="coerce")
            recorded_at = rec_dt.isoformat() if pd.notnull(rec_dt) else None
        except Exception:
            recorded_at = None

        # 为该机型容量下的每个颜色生成一条记录（套用 color-specific 调整）
        for _, it in sub.iterrows():
            part = _norm(it.get("part_number"))
            color = _norm(it.get("color"))
            if not part:
                continue
            adj = _apply_adjust_for_colorname(color, rules)
            price = base_price + adj
            if price <= 0:
                # 价格异常则跳过
                continue
            out_rows.append({
                "part_number": part,
                "shop_name": SHOP,
                "price_new": int(price),
                "recorded_at": recorded_at
            })

    if not out_rows:
        return pd.DataFrame(columns=["part_number","shop_name","price_new","recorded_at"])

    out = pd.DataFrame(out_rows, columns=["part_number","shop_name","price_new","recorded_at"])
    return out

In [50]:
de = _load_iphone17_info_df()

df_2 = pd.read_csv("/Users/syu/PycharmProjects/YamagotiProjects/shop2.csv")

res = clean_shop2(df_2)
res

   part_number         model_name  capacity_gb      color
0     MG674J/A          iPhone 17          256       ブラック
1     MG684J/A          iPhone 17          256       ホワイト
2     MG694J/A          iPhone 17          256     ミストブルー
3     MG6A4J/A          iPhone 17          256      ラベンダー
4     MG6C4J/A          iPhone 17          256        セージ
5     MG6D4J/A          iPhone 17          512       ブラック
6     MG6E4J/A          iPhone 17          512       ホワイト
7     MG6F4J/A          iPhone 17          512     ミストブルー
8     MG6G4J/A          iPhone 17          512      ラベンダー
9     MG6H4J/A          iPhone 17          512        セージ
10    MG854J/A      iPhone 17 Pro          256       シルバー
11    MG864J/A      iPhone 17 Pro          256  コズミックオレンジ
12    MG874J/A      iPhone 17 Pro          256    ディープブルー
13    MG894J/A      iPhone 17 Pro          512       シルバー
14    MG8A4J/A      iPhone 17 Pro          512  コズミックオレンジ
15    MG8C4J/A      iPhone 17 Pro          512    ディープブルー
16    MG8D4J/A

,part_number,shop_name,price_new,recorded_at
0,MFY84J/A,海峡通信,216000,2025-09-30T18:45:59+00:00
1,MFY94J/A,海峡通信,216000,2025-09-30T18:45:59+00:00
2,MFYA4J/A,海峡通信,215000,2025-09-30T18:45:59+00:00
3,MG854J/A,海峡通信,179000,2025-09-30T18:45:59+00:00
4,MG864J/A,海峡通信,179000,2025-09-30T18:45:59+00:00
5,MG874J/A,海峡通信,178000,2025-09-30T18:45:59+00:00
6,MG274J/A,海峡通信,146000,2025-09-30T18:45:59+00:00
7,MG284J/A,海峡通信,146000,2025-09-30T18:45:59+00:00
8,MG294J/A,海峡通信,146000,2025-09-30T18:45:59+00:00
9,MG2A4J/A,海峡通信,146000,2025-09-30T18:45:59+00:00
